In [1]:
import os
from autogen import ConversableAgent

In [3]:
def calculator(a,b):
    ''' 
    returns a/g , b/g where g is the gcd of a and b
    '''
    original_a = a
    original_b = b
    #compute gcd 
    while b!=0:
        a,b = b,a%b
    return original_a//a, original_b//a
    
    

In [4]:
cathy = ConversableAgent(
    "cathy",
    system_message="You are an expert mathematician who is seasoned at collaborating with others to solve problems and explain your solutions.",
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
    # function_map={'calculator' : calculator}
)

joe = ConversableAgent(
    "joe",
    system_message="You are an expert mathematician who is seasoned at revising other mathematician's solutions and explaining your solutions.",
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

In [5]:
result = joe.initiate_chat(cathy, message = "Cathy, let's solve this problem together: Suppose that we roll four 6-sided fair dice with faces numbered 1 to~6. Let $a/b$ be the probability that the highest roll is a 5, where $a$ and $b$ are relatively prime positive integers. Find $a + b$. Let's revise each other's answers until we get the right solution.", max_turns=2)

joe (to cathy):

Cathy, let's solve this problem together: Suppose that we roll four 6-sided fair dice with faces numbered 1 to~6. Let $a/b$ be the probability that the highest roll is a 5, where $a$ and $b$ are relatively prime positive integers. Find $a + b$. Let's revise each other's answers until we get the right solution.

--------------------------------------------------------------------------------
cathy (to joe):

Alright, let's start solving this problem:

First, let's find the total number of outcomes when we roll four 6-sided dice. Each roll has 6 possible outcomes, so rolling the dice four times gives us a total of $6^4$ outcomes.

Next, we have to calculate the probability that the highest roll is exactly 5. That means all the numbers we roll are in the range of 1 to 5 (as having a 6 would make the highest roll not 5). So, the number of outcomes here would be $5^4$.

But, if we stop here, we're also counting the cases where 5 does not appear at all (i.e., all rolls are 1

In [7]:
from openai import OpenAI
import json

client = OpenAI()

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
resonse = run_conversation()

In [8]:
resonse.choices[0].message.content

'Here is the current weather for the locations you asked about:\n\n- **San Francisco, CA**: 72°F\n- **Tokyo, Japan**: 10°C\n- **Paris, France**: 22°C\n\nLet me know if you need any more details!'